# 見積書自動作成

## 1. ライブラリ・データ読み込み

In [1]:
import pandas as pd
import openpyxl, pprint, os, sys
from datetime import datetime, timedelta
from openpyxl.styles import Alignment

In [2]:
# csvファイルを読み込む
products = pd.read_csv("../input/products.csv", encoding="shift_jis")
orders = pd.read_csv("../input/orders.csv", encoding="shift_jis")

In [3]:
products.head()

,product_id,product_name,unit_price,unit,note,tax_rate,category,supplier
0,P001,ノートパソコン,100000,台,15.6インチ SSD 512GB,10,PC本体,TechSupplier A
1,P002,レーザープリンタ,40000,台,両面印刷対応 A4,10,周辺機器,OfficeGear Inc.
2,P003,インクカートリッジ,3000,個,ブラック・カラー対応,10,消耗品,InkWorks
3,P004,HDMIケーブル,1200,本,1.5m 金メッキ仕様,10,周辺機器,CableMart
4,P005,USBメモリ 64GB,2000,本,USB 3.0対応,10,周辺機器,MemoryWorld


In [4]:
orders.head()

,order_id,customer_name,order_date,product_id,quantity,invoice_registration_number
0,O0001,高橋真由美,2025/5/10,P007,5,T1234567890001
1,O0001,高橋真由美,2025/5/10,P008,5,T1234567890001
2,O0001,高橋真由美,2025/5/10,P004,5,T1234567890001
3,O0002,鈴木健,2025/5/11,P010,1,T1234567890002
4,O0002,鈴木健,2025/5/11,P008,5,T1234567890002


In [5]:
# テンプレートを読み込む
quotation_wb = openpyxl.load_workbook("../template/quotation_template.xlsx")
# 対象のワークシートを読み込む
quotation_ws = quotation_wb["Quotation"]

## 2. 注文IDを入力し、対象のデータを取り出す

In [6]:
# 対象の注文IDを対話形式で入力
target_id = input("注文IDを入力して下さい")

注文IDを入力して下さいO0001


In [7]:
# 情報を元に見積書を作成
# 対象の取引データを抽出
target_orders = orders[orders["order_id"] == target_id]
target_orders

,order_id,customer_name,order_date,product_id,quantity,invoice_registration_number
0,O0001,高橋真由美,2025/5/10,P007,5,T1234567890001
1,O0001,高橋真由美,2025/5/10,P008,5,T1234567890001
2,O0001,高橋真由美,2025/5/10,P004,5,T1234567890001


In [8]:
# データを結合
merged = target_orders.merge(products, on="product_id", how="left")

In [9]:
# 金額列の小計
merged["subtotal"]  = merged["unit_price"] * merged["quantity"]

In [10]:
# 消費税
merged["VAT"] = merged["subtotal"] * merged["tax_rate"] / 100
merged["VAT"] = merged["VAT"].round(0).astype(int)

In [11]:
# 合計
merged["total_amount"] = merged["subtotal"] + merged["VAT"]
merged["total_amount"] = merged["total_amount"].astype(int)

## 3. 見積書作成、ファイルを出力

In [12]:
# 見積書への入力作業
# 明細の開始行
start_row = 19

# 右詰めスタイルを作成
right_align = Alignment(horizontal="right")

# 日付設定
today = datetime.today()
due_date = today + timedelta(days=30)

# 明細行描写
for i, (_, row) in enumerate(merged.iterrows()):
    # No(連番)
    quotation_ws.cell(row=start_row + i, column=2, value=i + 1) 
    # 商品名
    quotation_ws.cell(row=start_row + i, column=3, value=row["product_name"]) 
     # 数量
    quotation_ws.cell(row=start_row + i, column=4, value=row["quantity"])
    # 単価(円)
    cell = quotation_ws.cell(row=start_row + i, column=5, value=row["unit_price"]) 
    cell.number_format = '"¥"#,##0'
    # 金額(円)
    cell = quotation_ws.cell(row=start_row + i, column=6, value=row["subtotal"]) 
    cell.number_format = '"¥"#,##0'
    
# 小計金額
cell = quotation_ws.cell(row=35, column=6, value=merged["subtotal"].sum()) 
cell.number_format = '"¥"#,##0'

# 消費税
cell = quotation_ws.cell(row=36, column=6, value=merged["VAT"].sum()) 
cell.number_format = '"¥"#,##0'

# 合計金額
cell = quotation_ws.cell(row=37, column=6, value=merged["total_amount"].sum()) 
cell.number_format = '"¥"#,##0'

# 宛名
quotation_ws["B3"] = merged.iloc[0]["customer_name"] + "  様"

# 見積日
cell = quotation_ws.cell(row=12, column=6, value=today.strftime("%Y年%m月%d日")) 
cell.alignment = right_align

# 振込期限
cell = quotation_ws.cell(row=13, column=6, value=due_date.strftime("%Y年%m月%d日")) 
cell.alignment = right_align
    
quotation_wb.save(f"../output/{target_id}_quotation.xlsx")